本文通过对爬取到的 40 个已开通地铁的城市，共计5000+地铁站点进行数据分析，通过简单数据清洗、可视化作图、excel 三维地图等多个方式进行展示性分析


另外，在进行地铁数据分析的时候，需要特别注意爬取的数据集中数据的准确度

In [2]:
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings('ignore')

In [3]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 0.5px  black solid !important;
  color: black !important;
}


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
"""
sns 相关设置
@return:
"""
# 声明使用 Seaborn 样式
sns.set()
# 有五种seaborn的绘图风格，它们分别是：darkgrid, whitegrid, dark, white, ticks。默认的主题是darkgrid。
sns.set_style("whitegrid")
# 有四个预置的环境，按大小从小到大排列分别为：paper, notebook, talk, poster。其中，notebook是默认的。
sns.set_context('talk')
# 中文字体设置-黑体
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False


In [5]:
df_data = pd.read_csv('../data/全国城市地铁站点信息V202104（带经纬度）.csv', encoding='gbk')
df_data.sample(5)


,站点名称,POI编号,拼音名称,gd经度,gd纬度,路线名称,城市名称,行政区名称,bd经度,bd纬度
2279,花溪,BV10046023,HuaXi,106.550211,29.432287,3号线,重庆,巴南区,106.556689,29.438522
2772,富国街,BV10102321,fu guo jie,121.584703,38.898792,1号线,大连,沙河口区,121.591434,38.904431
2065,六合开发区,BV10052793,liu he kai fa qu,118.805334,32.305196,S8号线(宁天线),南京,六合区,118.811502,32.311537
4803,瓯华,BV11102063,OuHua,120.934880,27.930516,S1线,温州,洞头区,120.941554,27.936360
1401,凤凰城,BV10527878,FengHuangCheng,113.932831,22.747433,6号线,深圳,光明区,113.939407,22.753067


#### 数据清洗

删除完全重复的站点

类似于地铁环线、地铁一期与二期等都会出现重复站点，对应的重复站点数据是完全一致的，可以直接删除

In [6]:
"""删除完全重复的站点"""
df_data_1 = df_data.drop_duplicates()
df_data_1.sample(5)

,站点名称,POI编号,拼音名称,gd经度,gd纬度,路线名称,城市名称,行政区名称,bd经度,bd纬度
1210,石溪,BV10544498,shi xi,113.285951,23.067937,广佛线,广州,海珠区,113.292530,23.074206
213,海淀黄庄,BV10013411,hai dian huang zhuang,116.317564,39.975996,10号线,北京,海淀区,116.324346,39.981865
109,北运河东,BV10416651,bei yun he dong,116.707056,39.903268,6号线,北京,通州区,116.713760,39.909262
2473,向阳路,BV11472646,XiangYang Lu,120.442553,30.260328,1号线,杭州,萧山区,120.449181,30.266093
3514,西部汽车站,BV10544508,XiBu QiCheZhan,102.638836,25.035271,3号线,昆明,盘龙区,102.794882,25.042683


部分地铁站点因为地理位置对不上而出现空值，可以看一下具体情况：


In [7]:
print(df_data_1[df_data_1['行政区名称'].isnull()].shape)
df_data_1[df_data_1['行政区名称'].isnull()].sample(5)


(51, 10)


,站点名称,POI编号,拼音名称,gd经度,gd纬度,路线名称,城市名称,行政区名称,bd经度,bd纬度
2127,坚尼地城,BV10420638,JianNi DiCheng,114.133869,22.278567,港岛线,香港,NaN,114.182081,22.277594
2129,西营盘,BV10420639,XiYingPan,114.147572,22.282666,港岛线,香港,NaN,NaN,NaN
2126,香港,BV10313841,XiangGang,114.162952,22.282068,东涌线,香港,NaN,NaN,NaN
2221,尖沙咀,BV10313829,JianShaZui,114.177190,22.294678,荃湾线,香港,NaN,114.192193,22.273526
2145,何文田,BV10789130,HeWenTian,114.187751,22.306628,观塘线,香港,NaN,114.187375,22.315758


一共 51 个数据为空，香港数据为空的较多，其他城市只有较少数，可以单独处理，或者手动补充一下数据

In [8]:
df_data_notnull = df_data_1[df_data_1['行政区名称'].notnull()]
df_data_notnull.sample(5)

,站点名称,POI编号,拼音名称,gd经度,gd纬度,路线名称,城市名称,行政区名称,bd经度,bd纬度
4162,硕放机场,BV10128134,ShuoFang JiChang,120.430807,31.502584,3号线,无锡,新吴区,120.440156,31.508856
3069,人民北路,BV10061673,RenMin BeiLu,104.073056,30.684833,6号线,成都,金牛区,104.079208,30.691592
1160,钟落潭,BV10851527,zhong luo tan,113.400686,23.375649,14号线,广州,白云区,113.407386,23.382315
4460,安吉客运站,BV10730199,AnJi KeYunZhan,108.297843,22.879757,3号线,南宁,西乡塘区,108.305817,22.884455
1549,佛祖岭,BV10890079,fo zu ling,114.439777,30.438671,2号线,武汉,江夏区,114.446341,30.444579


一共 4808 条数据，表示有 4808 个地铁站点，每个站点都有 10 个详细的字段。


In [9]:
df_data_2 = df_data_notnull.copy()
# 计算站点的换乘个数
df_address_cnt = df_data_2.groupby(['城市名称', '站点名称']).agg({'拼音名称':'count'}).reset_index().rename(columns={'拼音名称': '换乘站点'})
df_data_3 = df_data_2.merge(df_address_cnt, on=['城市名称', '站点名称'], how='left')
df_data_3.sample(5)


,站点名称,POI编号,拼音名称,gd经度,gd纬度,路线名称,城市名称,行政区名称,bd经度,bd纬度,换乘站点
4021,双山,BV10425292,ShuangShan,120.397406,36.114280,3号线,青岛,市北区,120.403728,36.120132,1
1825,洞庭路,BV10990199,dong ting lu,117.245877,39.043590,6号线,天津,河西区,117.251290,39.050160,1
1619,建安街,BV10575213,jian an jie,114.313759,30.504025,7号线,武汉,武昌区,114.320362,30.509791,1
2496,青山湖,BV11280860,QingShanHu,119.755663,30.255795,16号线,杭州,临安区,119.762787,30.261856,1
3349,苏家塘,BV11300054,SuJiaTang,102.692655,25.063907,4号线,昆明,五华区,102.700200,25.069696,1


换乘站点列表示一个地铁站有多少个换乘站。

例如：金安桥站是S1线和6号线的换乘点，所以对应的换乘站点为2

In [10]:
df_data_3[df_data_3['站点名称']=='金安桥']


,站点名称,POI编号,拼音名称,gd经度,gd纬度,路线名称,城市名称,行政区名称,bd经度,bd纬度,换乘站点
0,金安桥,BV10813545,jin an qiao,116.163167,39.92362,S1线,北京,石景山区,116.169034,39.929669,2
137,金安桥,BV10813545,jin an qiao,116.163167,39.92362,6号线,北京,石景山区,116.169034,39.929669,2



数据前期清洗大概就这些

另外，此次的数据仍存在部分异常，感兴趣的可以做一个深度清洗，再进行下一步的分析

#### 整体城市分析

先来看一下40城的地铁站点数量

In [10]:
df_data_eda = df_data_3.copy()
"""查看城市的地铁站点数量"""
df_city_cnt = df_data_eda.groupby('城市名称').agg({'站点名称':pd.Series.nunique}).reset_index().rename(columns={'站点名称': 'metro_cnt'})
df_city_cnt.sample(5)


,城市名称,metro_cnt
38,青岛,106
30,福州,46
28,温州,18
32,西安,153
4,兰州,20


In [15]:
# 设置标题
plt.figure(figsize=(30, 8))
plt.title('全国40城市地铁站点数量分布')
ax = sns.barplot(data=df_city_cnt, x='城市名称', y='metro_cnt')
plt.ylabel('地铁站点数量')

# 显示数据的具体数值
for x, y in zip(range(0, len(df_city_cnt['城市名称'].index.tolist())), df_city_cnt['metro_cnt'].values.tolist()):
    ax.text(x-0.3, y+0.5, '%d' % y, color='black')
    
plt.show()


<Figure size 2160x576 with 1 Axes>

可以看到，地铁站点最多的城市是上海，数量是380；其次分别是：北京、成都、广州、深圳、武汉、重庆、杭州 等

<br>

再来看每个城市的地铁线路个数

In [11]:
"""查看每个城市的地铁线路个数"""
df_metro = df_data_eda[df_data_eda['换乘站点']==1].groupby('城市名称')
df_metro_cnt = df_data_eda.groupby('城市名称').agg({'路线名称':pd.Series.nunique}).reset_index().rename(columns={'路线名称': 'road_cnt'})
df_metro_cnt.sample(5)


,城市名称,road_cnt
4,兰州,1
21,无锡,3
0,上海,19
29,石家庄,3
7,南宁,4


In [17]:
# 设置标题
plt.figure(figsize=(30, 8))
plt.title('全国40城市地铁线路数量分布')
ax = sns.barplot(data=df_metro_cnt, x='城市名称', y='road_cnt')
plt.ylabel('地铁线路数量')

# 显示数据的具体数值
for x, y in zip(range(0, len(df_metro_cnt['城市名称'].index.tolist())), df_metro_cnt['road_cnt'].values.tolist()):
    ax.text(x-0.3, y+0.5, '%d' % y, color='black')
    
plt.show()


<Figure size 2160x576 with 1 Axes>

其中北京的地铁线路最多，达到24条，其次分别是上海、广州、成都、香港、深圳、武汉、重庆等

在北京搬砖的同学应该了解，北京有很多新开通的地铁线路是以地区命名的，而且有的是分南段、北段，这样在统计的时候根据名称自然是会当做很多条线。

像深圳新开通的 8 号线（2号线延长线），高德地图将其归为了 2 号线，所以深圳对应的地铁总线路就只有 10 条，而不是 11 条

#### 针对TOP城市进行分析

本来打算只针对北上广深做进一步分析，但是突然拔尖的成都让我很感兴趣，于是决定一起分析了

In [18]:
df_data_sz = df_data_eda[df_data_eda['城市名称']=='深圳']
df_data_gz = df_data_eda[df_data_eda['城市名称']=='广州']
df_data_sh = df_data_eda[df_data_eda['城市名称']=='上海']
df_data_bj = df_data_eda[df_data_eda['城市名称']=='北京']
df_data_cd = df_data_eda[df_data_eda['城市名称']=='成都']

print(df_data_bj.shape)
print(df_data_sh.shape)
print(df_data_gz.shape)
print(df_data_sz.shape)
print(df_data_cd.shape)


(414, 11)
(468, 11)
(281, 11)
(283, 11)
(333, 11)


In [19]:
"""查看深圳换乘站点情况"""
df_sz_hc_temp = df_data_sz[df_data_sz['换乘站点']>1].groupby('站点名称')['路线名称'].agg(len).reset_index().rename(columns={'路线名称': '换乘站点'})
df_sz_hc = df_sz_hc_temp.groupby('换乘站点').agg(len).reset_index().rename(columns={'站点名称': '地铁路线'})
df_sz_hc

,换乘站点,地铁路线
0,2,36
1,3,4
2,4,1


4 站换乘只有 1个，3站换乘有4个，2站换乘有36个

对应的，其他城市的可以通过可视化更直观的体现

In [20]:
"""查看城市的换乘站点数量"""
df_gz_hc_temp = df_data_gz[df_data_gz['换乘站点']>1].groupby('站点名称').agg({'路线名称':'count'}).reset_index().rename(columns={'路线名称': '换乘站点'})
df_gz_hc = df_gz_hc_temp.groupby('换乘站点').agg(len).reset_index().rename(columns={'站点名称': '地铁路线'})

df_gz_hc_temp = df_data_gz[df_data_gz['换乘站点']>1].groupby('站点名称').agg({'路线名称':'count'}).reset_index().rename(columns={'路线名称': '换乘站点'})
df_gz_hc = df_gz_hc_temp.groupby('换乘站点').agg(len).reset_index().rename(columns={'站点名称': '地铁路线'})

df_sh_hc_temp = df_data_sh[df_data_sh['换乘站点']>1].groupby('站点名称').agg({'路线名称':'count'}).reset_index().rename(columns={'路线名称': '换乘站点'})
df_sh_hc = df_sh_hc_temp.groupby('换乘站点').agg(len).reset_index().rename(columns={'站点名称': '地铁路线'})

df_bj_hc_temp = df_data_bj[df_data_bj['换乘站点']>1].groupby('站点名称').agg({'路线名称':'count'}).reset_index().rename(columns={'路线名称': '换乘站点'})
df_bj_hc = df_bj_hc_temp.groupby('换乘站点').agg(len).reset_index().rename(columns={'站点名称': '地铁路线'})

df_cd_hc_temp = df_data_cd[df_data_cd['换乘站点']>1].groupby('站点名称').agg({'路线名称':'count'}).reset_index().rename(columns={'路线名称': '换乘站点'})
df_cd_hc = df_cd_hc_temp.groupby('换乘站点').agg(len).reset_index().rename(columns={'站点名称': '地铁路线'})


In [23]:
fig, ax = plt.subplots(2, 3, figsize=(27, 21))
sns.barplot(data=df_bj_hc, x='换乘站点', y='地铁路线', ax=ax[0, 0])
sns.barplot(data=df_sh_hc, x='换乘站点', y='地铁路线', ax=ax[0, 1])
sns.barplot(data=df_gz_hc, x='换乘站点', y='地铁路线', ax=ax[0, 2])
sns.barplot(data=df_sz_hc, x='换乘站点', y='地铁路线', ax=ax[1, 0])
sns.barplot(data=df_cd_hc, x='换乘站点', y='地铁路线', ax=ax[1, 1])
# 设置标题
ax[0, 0].set_title('北京市换乘站点的分布图')
ax[0, 1].set_title('上海市换乘站点的分布图')
ax[0, 2].set_title('广州市换乘站点的分布图')
ax[1, 0].set_title('深圳市换乘站点的分布图')
ax[1, 1].set_title('成都市换乘站点的分布图')

# 显示数据的具体数值
for x, y in zip(range(0, len(df_bj_hc['地铁路线'].index.tolist())), df_bj_hc['地铁路线'].values.tolist()):
    ax[0, 0].text(x, y+0.5, '%d' % y, color='black')
for x, y in zip(range(0, len(df_sh_hc['地铁路线'].index.tolist())), df_sh_hc['地铁路线'].values.tolist()):
    ax[0, 1].text(x, y+0.5, '%d' % y, color='black')
for x, y in zip(range(0, len(df_gz_hc['地铁路线'].index.tolist())), df_gz_hc['地铁路线'].values.tolist()):
    ax[0, 2].text(x, y+0.5, '%d' % y, color='black')
for x, y in zip(range(0, len(df_sz_hc['地铁路线'].index.tolist())), df_sz_hc['地铁路线'].values.tolist()):
    ax[1, 0].text(x, y+0.5, '%d' % y, color='black')
for x, y in zip(range(0, len(df_cd_hc['地铁路线'].index.tolist())), df_cd_hc['地铁路线'].values.tolist()):
    ax[1, 1].text(x, y+0.5, '%d' % y, color='black')

plt.show()

<Figure size 1944x1512 with 6 Axes>

由图可知，深圳拥有1个四站换乘的站点，上海拥有2个四站换乘的站点

不愧是新崛起的超一线城市，看四站换乘就能看出一二

<br>

可能四站换乘站点还有其他的，一起来看一看：

In [24]:
# 深圳和上海的四线换乘站点分别如下：
df_data_eda[(df_data_eda['换乘站点']==4) & (df_data_eda['城市名称'].isin(['上海', '深圳']))]


,站点名称,POI编号,拼音名称,gd经度,gd纬度,路线名称,城市名称,行政区名称,bd经度,bd纬度,换乘站点
453,龙阳路,BV10039851,LongYang Lu,121.557634,31.203575,2号线,上海,浦东新区,121.563934,31.209171,4
456,世纪大道,BV10039835,ShiJi DaDao,121.527213,31.228682,2号线,上海,浦东新区,121.533948,31.234503,4
516,世纪大道,BV10039835,ShiJi DaDao,121.527213,31.228682,4号线,上海,浦东新区,121.533948,31.234503,4
557,世纪大道,BV10039835,ShiJi DaDao,121.527213,31.228682,6号线,上海,浦东新区,121.533948,31.234503,4
575,龙阳路,BV10039851,LongYang Lu,121.557634,31.203575,7号线,上海,浦东新区,121.563934,31.209171,4
647,世纪大道,BV10039835,ShiJi DaDao,121.527213,31.228682,9号线,上海,浦东新区,121.533948,31.234503,4
840,龙阳路,BV10039851,LongYang Lu,121.557634,31.203575,16号线,上海,浦东新区,121.563934,31.209171,4
875,龙阳路,BV10039851,LongYang Lu,121.557634,31.203575,磁悬浮,上海,浦东新区,121.563934,31.209171,4
1173,车公庙,BV10249928,che gong miao,114.025830,22.536247,1号线/罗宝线,深圳,福田区,114.033650,22.541996,4
1355,车公庙,BV10249928,che gong miao,114.025830,22.536247,7号线/西丽线,深圳,福田区,114.033650,22.541996,4


图中的四站换乘分别是：深圳的车公庙站，上海的世纪大道、龙阳路站，其中龙阳路站是2号线、7号线、16号线和磁悬浮线四站换乘

In [25]:
# 除此之外其他城市的四线换乘站点分别如下：
df_data_eda[(df_data_eda['换乘站点']==4) & ~(df_data_eda['城市名称'].isin(['上海', '深圳']))]


,站点名称,POI编号,拼音名称,gd经度,gd纬度,路线名称,城市名称,行政区名称,bd经度,bd纬度,换乘站点
1856,南京南站,BV10053672,nan jing nan zhan,118.796786,31.970968,S1号线(机场线),南京,雨花台区,118.804341,31.975024,4
1868,南京南站,BV10053672,nan jing nan zhan,118.796786,31.970968,1号线,南京,雨花台区,118.804341,31.975024,4
1928,南京南站,BV10053672,nan jing nan zhan,118.796786,31.970968,S3号线(宁和线),南京,雨花台区,118.804341,31.975024,4
1936,南京南站,BV10053672,nan jing nan zhan,118.796786,31.970968,3号线,南京,雨花台区,118.804341,31.975024,4
2647,开发区,BV10104043,kai fa qu,121.775000,39.053600,3号线,大连,金州区,121.780941,39.059644,4
2667,开发区,BV10104043,kai fa qu,121.775000,39.053600,保税区线,大连,金州区,121.780941,39.059644,4
2675,开发区,BV10104043,kai fa qu,121.775000,39.053600,九里线,大连,金州区,121.780941,39.059644,4
2689,开发区,BV10104043,kai fa qu,121.775000,39.053600,九里支线,大连,金州区,121.780941,39.059644,4


除图中之外，四站换乘的地铁站还有：南京的南京南站，大连的开发区站

其中大连的开发区站并不能算做真实意义上的4站换乘，看图就知：


![Image Name](https://cdn.kesci.com/upload/image/qrm4ttfv6o.png?imageView2/0/w/960/h/960)




#### 针对某一城市做具体分析

##### 北京市分析

对北京市各区域的站点数量进行深度分析，如下：

In [26]:
"""查看城市的地铁站点数量"""
df_bj_area_cnt = df_data_bj.groupby('行政区名称').agg({'站点名称':pd.Series.nunique}).reset_index().rename(columns={'站点名称': '地铁站点数'})
df_bj_area_cnt


,行政区名称,地铁站点数
0,东城区,28
1,丰台区,42
2,大兴区,20
3,广阳区,1
4,房山区,18
5,昌平区,20
6,朝阳区,82
7,海淀区,63
8,石景山区,7
9,西城区,29


In [30]:
# 设置标题
plt.figure(figsize=(25, 10))
plt.title('北京市各区域地铁站点数量的分布图')
ax = sns.barplot(data=df_bj_area_cnt, x='行政区名称', y='地铁站点数')
plt.ylabel('地铁站点数量')

# 显示数据的具体数值
for x, y in zip(range(0, len(df_bj_area_cnt['地铁站点数'].index.tolist())), df_bj_area_cnt['地铁站点数'].values.tolist()):
    ax.text(x, y, '%d' % y, color='black')
plt.show()


<Figure size 1800x720 with 1 Axes>

其中，朝阳区地铁站点数量为82，,海淀区地铁站点数量为63

顺义区、石景山区、门头沟区、广阳区地铁站点数量较少，地铁能辐射到的区域较少。


##### 上海市分析

对上海市各区域的站点数量进行深度分析，如下：

In [32]:
"""查看城市的地铁站点数量"""
df_sh_area_cnt = df_data_sh.groupby('行政区名称').agg({'站点名称':pd.Series.nunique}).reset_index().rename(columns={'站点名称': '地铁站点数'})
df_sh_area_cnt


,行政区名称,地铁站点数
0,嘉定区,14
1,奉贤区,7
2,宝山区,33
3,崇明区,1
4,徐汇区,32
5,普陀区,24
6,杨浦区,24
7,松江区,9
8,浦东新区,114
9,虹口区,12


In [41]:
# 设置标题
plt.figure(figsize=(25, 10))
plt.title('上海市各区域地铁站点数量的分布图')
ax = sns.barplot(data=df_sh_area_cnt, x='行政区名称', y='地铁站点数')
plt.ylabel('地铁站点数量')

# 显示数据的具体数值
for x, y in zip(range(0, len(df_sh_area_cnt['地铁站点数'].index.tolist())), df_sh_area_cnt['地铁站点数'].values.tolist()):
    ax.text(x, y, '%d' % y, color='black')
plt.show()

<Figure size 1800x720 with 1 Axes>

其中，浦江新区地铁站点数量为114，遥遥领先其他各区。

第二名的闵行区地铁站点数量为46，崇明区只有1个地铁站点，区域被辐射到的区域较少。

蒲江新区站点多的原因主要是因为面积大，网上找了一张图，看了你就明白了：



![Image Name](https://cdn.kesci.com/upload/image/qrm4xhho29.png?imageView2/0/w/960/h/960)



##### 广州市分析

对广州市各区域的站点数量进行深度分析，如下：

In [34]:
"""查看城市的地铁站点数量"""
df_gz_area_cnt = df_data_gz.groupby('行政区名称').agg({'站点名称':pd.Series.nunique}).reset_index().rename(columns={'站点名称': '地铁站点数'})
df_gz_area_cnt


,行政区名称,地铁站点数
0,从化区,5
1,南沙区,13
2,增城区,12
3,天河区,40
4,海珠区,26
5,番禺区,25
6,白云区,35
7,花都区,15
8,荔湾区,20
9,越秀区,20


In [42]:
# 设置标题
plt.figure(figsize=(25, 10))
plt.title('广州市各区域地铁站点数量的分布图')
ax = sns.barplot(data=df_gz_area_cnt, x='行政区名称', y='地铁站点数')
plt.ylabel('地铁站点数量')

# 显示数据的具体数值
for x, y in zip(range(0, len(df_gz_area_cnt['地铁站点数'].index.tolist())), df_gz_area_cnt['地铁站点数'].values.tolist()):
    ax.text(x, y, '%d' % y, color='black')
plt.show()

<Figure size 1800x720 with 1 Axes>

其中，天河区、白云区、黄埔区的地铁站点数量依次为 40、35、31，基本分布很均匀

从化区的地铁站点数量最少，地铁能辐射到的区域较少。

##### 深圳市分析

对深圳市各区域的站点数量进行深度分析，如下：

In [36]:
"""查看城市的地铁站点数量"""
df_sz_area_cnt = df_data_sz.groupby('行政区名称').agg({'站点名称':pd.Series.nunique}).reset_index().rename(columns={'站点名称': '地铁站点数'})
df_sz_area_cnt


,行政区名称,地铁站点数
0,光明区,10
1,南山区,49
2,宝安区,30
3,盐田区,5
4,福田区,54
5,罗湖区,29
6,龙华区,21
7,龙岗区,38


In [43]:
# 设置标题
plt.figure(figsize=(25, 10))
plt.title('深圳市各区域地铁站点数量的分布图')
ax = sns.barplot(data=df_sz_area_cnt, x='行政区名称', y='地铁站点数')
plt.ylabel('地铁站点数量')

# 显示数据的具体数值
for x, y in zip(range(0, len(df_sz_area_cnt['地铁站点数'].index.tolist())), df_sz_area_cnt['地铁站点数'].values.tolist()):
    ax.text(x, y, '%d' % y, color='black')
plt.show()

<Figure size 1800x720 with 1 Axes>

其中，福田区、南山区的地铁站点数量分别为：54和49，领先其他各区

盐田区站点数量较少，盲猜是新开通的8号线。

##### 成都市分析

对成都市各区域的站点数量进行深度分析，如下：

In [39]:
"""查看城市的地铁站点数量"""
df_cd_area_cnt = df_data_cd.groupby('行政区名称').agg({'站点名称':pd.Series.nunique}).reset_index().rename(columns={'站点名称': '地铁站点数'})
df_cd_area_cnt


,行政区名称,地铁站点数
0,双流区,61
1,成华区,28
2,新津区,7
3,新都区,14
4,武侯区,52
5,温江区,12
6,简阳市,2
7,郫都区,13
8,金牛区,40
9,锦江区,20


In [44]:
# 设置标题
plt.figure(figsize=(25, 10))
plt.title('成都市各区域地铁站点数量的分布图')
ax = sns.barplot(data=df_cd_area_cnt, x='行政区名称', y='地铁站点数')
plt.ylabel('地铁站点数量')

# 显示数据的具体数值
for x, y in zip(range(0, len(df_cd_area_cnt['地铁站点数'].index.tolist())), df_cd_area_cnt['地铁站点数'].values.tolist()):
    ax.text(x, y, '%d' % y, color='black')
plt.show()


<Figure size 1800x720 with 1 Axes>

其中，双流区、武侯区、金牛区的地铁站点数量分别为：61、52和40，遥遥领先其他各区

没有想到的是，成都的地铁站点比深圳多，而且成都的区域聚集效应更明显。

##### 城市地图可视化 

最后，通过 excel 的三维地图再来看看上面5个城市的聚集效应到底如何


北京市各区域地铁站点情况三维地图展示：


![Image Name](https://cdn.kesci.com/upload/image/qrm57hqgno.png)

真的是规规整整，向四面八方进行辐射，圈状的布局非常明显


上海市各区域地铁站点情况三维地图展示：



![Image Name](https://cdn.kesci.com/upload/image/qrm5bvgf2e.png?imageView2/0/w/960/h/960)


上海确实是浦江新区的地铁站点相当多，整体聚集效应比较明显，中间圆心位置地理优势很突出


广州市各区域地铁站点情况三维地图展示：


![Image Name](https://cdn.kesci.com/upload/image/qrm5g15839.png?imageView2/0/w/960/h/960)



广州的交通现在辐射的已经很多了，像图中左下角的广佛线连接佛山，正下角的连接东莞，甚至以后会直接和深圳用轻轨连起来等，交通枢纽位置明显。

但是同样它的整体聚集效应也比较明显，通过柱子的高度就能看出来。

深圳市各区域地铁站点情况三维地图展示：



![Image Name](https://cdn.kesci.com/upload/image/qrm5n2r0li.png?imageView2/0/w/960/h/960)


深圳的核心区域面积比较小，导致其能辐射的区域有限，对应的只能往上半图辐射连接东莞，右上角辐射连接惠州。

> 这也正是站点数量被成都挤下去的原因

.

深圳主要是两个核心区：红色的福田区和亮黄色的南山区，中心区域分布比较均匀

大胆预测一下：按照深圳的发展速度，上半张图大概率会崛起


成都市各区域地铁站点情况三维地图展示：


![Image Name](https://cdn.kesci.com/upload/image/qrm5wj3a2a.png?imageView2/0/w/960/h/960)


成都是一个我没想到的城市，没想到它的交通会排在深圳前面，看图就能得出这得益于它的区域位置优越。

成都基本上各条线路已经辐射开了，后面可能会形成一个大的外部环线，其实，类似的城市还有一个，不知道大家能不能联想到？


ok，以上就是本节分析的全部内容。

感兴趣的同学可以关注我的专栏 👉[小一的学习笔记](https://www.heywhale.com/home/column/601fe851b23344001580c1f0)  方便第一时间获取更新的项目


.

#### 拓展

最后，大家可以根据如下角度进行更深入的分析：
- 通过客流量数据，结合地铁站点进行日内用户轨迹分析
- 通过租房数据，结合距离进行房价的进一步探索
- 通过地产数据，结合地铁站点进行价值分析


![Image Name](https://cdn.kesci.com/upload/image/qrhvm4qcxl.jpg?imageView2/0/w/960/h/960)